In [86]:
import puffin.tardis_importer as ti
symbol = 'BTC'
spot_trades = ti.generate_trade_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_trades_2021-10-28_{symbol}-USD.csv.gz')
spot_tobs = ti.generate_tob_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_quotes_2021-10-28_{symbol}-USD.csv.gz')
future_trades = ti.generate_trade_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_trades_2021-10-28_{symbol}-PERP.csv.gz')
future_tobs = ti.generate_tob_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_quotes_2021-10-28_{symbol}-PERP.csv.gz')

In [90]:
spot_tobs['timestamp'] = spot_tobs['exchange_timestamp_nanos'] // 10000000000
spot_tobs = spot_tobs.drop_duplicates(subset='timestamp', keep='last')
spot_trades['timestamp'] = spot_trades['exchange_timestamp_nanos'] // 10000000000
givens = spot_trades[spot_trades['given']]
givens = givens[['timestamp', 'price']].groupby(by=['timestamp']).min()
paids = spot_trades[~spot_trades['given']]
paids = paids[['timestamp', 'price']].groupby(by=['timestamp']).max()
bids = spot_tobs[['timestamp', 'bid_price']]
bids = bids.set_index('timestamp')
asks = spot_tobs[['timestamp', 'ask_price']]
asks = asks.set_index('timestamp')

In [91]:
import pandas as pd
left = pd.merge(bids, givens, how='left', left_index=True, right_index=True)
left['how_deep'] = left['bid_price'] - left['price']
left = left['how_deep'].value_counts(dropna=False)
left = left.sort_index(ascending=False).to_frame().cumsum()
left['how_deep'] = left['how_deep'].astype('float')
left['how_deep'] = left['how_deep'].div(left['how_deep'].iloc[-1])
left = left[left.index >= 0]
left

,how_deep
532.0,0.000116
512.0,0.000231
510.0,0.000347
475.0,0.000463
446.0,0.000579
...,...
4.0,0.343866
3.0,0.365625
2.0,0.387037
1.0,0.417130


In [92]:
import pandas as pd
right = pd.merge(asks, paids, how='left', left_index=True, right_index=True)
right['how_deep'] = right['price'] - right['ask_price']
right = right['how_deep'].value_counts(dropna=False)
right = right.sort_index(ascending=False).to_frame().cumsum()
right['how_deep'] = right['how_deep'].astype('float')
right['how_deep'] = right['how_deep'].div(right['how_deep'].iloc[-1])
right = right[right.index >= 0]
right

,how_deep
1332.0,0.000116
782.0,0.000231
404.0,0.000347
373.0,0.000463
341.0,0.000579
...,...
4.0,0.347801
3.0,0.371875
2.0,0.392940
1.0,0.421181
